In [ ]:
import os
import cv2
import json
import random
import colorsys

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from   keras.models               import Sequential, Model ,model_from_yaml ,load_model
from   keras.activations          import linear
from   keras.layers.normalization import BatchNormalization
from   keras.optimizers           import Adagrad
from   keras.utils                import plot_model
from   keras.engine.topology      import Layer
from   yolo3.model                import yolo_eval
from   keras                      import backend as K

font = cv2.FONT_HERSHEY_SIMPLEX
# Generate colors for drawing bounding boxes.



In [ ]:
def _get_class():
    with open(classes_path) as f:
        class_names = f.readlines()
    class_names = [c.strip() for c in class_names]
    return class_names

In [ ]:
def _get_anchors():
    with open(anchors_path) as f:
        anchors = f.readline()
        anchors = [float(x) for x in anchors.split(',')]
        anchors = np.array(anchors).reshape(-1, 2)
    return anchors

In [ ]:
def generate():
    assert model_path.endswith('.h5'), 'Keras model must be a .h5 file.'
    Infer_model = load_model(model_path)
    print('{} model, anchors, and classes loaded.'.format(model_path))

#     yaml_file = open('infer_model.yaml', 'r')
#     loaded_model_yaml = yaml_file.read()
#     yaml_file.close()
#    Infer_model = model_from_yaml(loaded_model_yaml)
    # Generate output tensor targets for filtered bounding boxes.
    input_image_shape = K.placeholder(shape=(2, ))
    boxes, scores, classes = yolo_eval(Infer_model.output, anchors,
                                       len(class_names), input_image_shape,
                                       score_threshold=score, iou_threshold=iou)
    return Infer_model,boxes, scores, classes,input_image_shape

In [ ]:
def close_session(self):
    sess.close()

In [ ]:
def detect_image(image,Infer_model,input_image_shape, animals=1, predictEdge=0, pos=-1, database=None):
    predictEdge /= 100
    if is_fixed_size:
        assert model_image_size[0] % 32 == 0, 'Multiples of 32 required'
        assert model_image_size[1] % 32 == 0, 'Multiples of 32 required'
        boxed_image = cv2.resize(image,model_image_size)
    else:
        new_image_size = (image.width - (image.width % 32),
                          image.height - (image.height % 32))
        boxed_image = letterbox_image(image, new_image_size)
    image_data = np.array(boxed_image, dtype='float32')

    # print(image_data.shape)
    image_data /= 255.
    image_data = np.expand_dims(image_data, 0)  # Add batch dimension.
    out_boxes, out_scores, out_classes = sess.run(
        [boxes, scores, classes],
        feed_dict={
            Infer_model.input: image_data,
            input_image_shape: [boxed_image.shape[0], boxed_image.shape[1]],
            K.learning_phase(): 0
        })

#    print('Found {} boxes for {}'.format(len(out_boxes), 'img'))

    thickness = (image.shape[0] + image.shape[1]) // 300

    for i, c in reversed(list(enumerate(out_classes))):
        predicted_class = class_names[c]
        box = out_boxes[i]
        score = out_scores[i]
        score = float(score)
        score = round(score, 2)
#        print(score, predicted_class, c ,box)
#        if((score < predictEdge) or ((animals >> c) & 1 == 0)):
#            continue
        label = '{} {:.2f}'.format(predicted_class, score)
        label_size = 10

        top, left, bottom, right = box
        top = max(0, np.floor(top + 0.5).astype('int32'))
        left = max(0, np.floor(left + 0.5).astype('int32'))
        bottom = min(image.shape[0], np.floor(bottom + 0.5).astype('int32'))
        right = min(image.shape[1], np.floor(right + 0.5).astype('int32'))
        # print(label, (left, top), (right, bottom), pos, c, type(top))
        if(pos != -1):
            database.insert({"pos": pos, "left": int(left), "right": int(right),
                             "top": int(top), "bottom": int(bottom), "predict": int(c), "score": score})
        if top - label_size >= 0:
            text_origin = np.array([left, top - label_size])
        else:
            text_origin = np.array([left, top + 1])
        cv2.rectangle(boxed_image,(left,top),(right,bottom),colors[c],3)    
        cv2.putText(boxed_image,label,tuple(text_origin),font,1,colors[c])
    # end = time.time()
    # print(end - start)
    return boxed_image

In [ ]:
model_path   = 'yolov3_weights.h5'
anchors_path = 'model_data/yolo_anchors.txt'
classes_path = 'model_data/coco_classes.txt'
score = 0.3
iou = 0.5
class_names = _get_class()
anchors = _get_anchors()
sess = K.get_session()
hsv_tuples = [(x / len(class_names), 1., 1.)
              for x in range(len(class_names))]
colors = list(map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples))
colors = list(
    map(lambda x: (int(x[0] * 255), int(x[1] * 255), int(x[2] * 255)),
        colors))
random.seed(10101)  # Fixed seed for consistent colors across runs.
# Shuffle colors to decorrelate adjacent classes.
random.shuffle(colors)
random.seed(None)  # Reset seed to default.
model_image_size = (416, 416)  # fixed size or (None, None)
is_fixed_size = model_image_size != (None, None)
Infer_model,boxes, scores, classes,input_image_shape = generate()

In [ ]:
image = cv2.imread("./test2.jpg",cv2.IMREAD_COLOR)
det_image=detect_image(image,Infer_model,input_image_shape)
plt.figure(figsize=(416,416))
plt.imshow(det_image)